# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [154]:
Energy = pd.read_excel(r'assets/Energy Indicators.xls', skiprows=17, skipfooter=38).drop(columns=['Unnamed: 0', 'Unnamed: 1'])
Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy.replace('...', np.NaN, inplace=True)
Energy['Energy Supply'] = Energy['Energy Supply'] * (10 ** 6)
Energy['Country'] = Energy['Country'].str.replace(r' \(.*\)', '', regex=True).replace(r'\d+$', '', regex= True)
Energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States",
                "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
############################

GDP = pd.read_csv(r'assets/world_bank.csv', skiprows=4)
GDP['Country Name'].replace({
"Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
, inplace=True)
GDP.rename({'Country Name' : 'Country'}, axis='columns', inplace=True)
############################

ScimEn = pd.read_excel(r'assets/scimagojr-3.xlsx')

dfs = [ScimEn, Energy, GDP]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Country', how='outer'), dfs)
merged_df

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,1.0,China,127050.0,126767.0,597237.0,411683.0,4.70,138.0,1.271910e+11,93.0,...,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
1,2.0,United States,96661.0,94747.0,792274.0,265436.0,8.20,230.0,9.083800e+10,286.0,...,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
2,3.0,Japan,30504.0,30287.0,223024.0,61554.0,7.31,134.0,1.898400e+10,149.0,...,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
3,4.0,United Kingdom,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7.920000e+09,124.0,...,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
4,5.0,Russian Federation,18534.0,18301.0,34266.0,12422.0,1.85,57.0,3.070900e+10,214.0,...,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,NaN,Vietnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.130761e+10,9.781739e+10,1.033556e+11,1.089346e+11,1.159317e+11,1.231662e+11,1.296292e+11,1.366576e+11,1.448347e+11,1.545086e+11
315,NaN,West Bank and Gaza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.016675e+09,7.878227e+09,7.201473e+09,8.709518e+09,8.913100e+09,9.611505e+09,1.100864e+10,1.053747e+10,1.042246e+10,1.171414e+10
316,NaN,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.023536e+13,6.282865e+13,6.398692e+13,6.291271e+13,6.564782e+13,6.770244e+13,6.937886e+13,7.104232e+13,7.291072e+13,7.470901e+13
317,NaN,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.672565e+10,2.761787e+10,2.872656e+10,2.991436e+10,3.090675e+10,2.624342e+10,2.689160e+10,2.800914e+10,NaN,NaN


In [2]:
def answer_one():
    # YOUR CODE HERE
    Energy = pd.read_excel(r'assets/Energy Indicators.xls', skiprows=17, skipfooter=38).drop(columns=['Unnamed: 0', 'Unnamed: 1'])
    Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    Energy.replace('...', np.NaN, inplace=True)
    Energy['Energy Supply'] = Energy['Energy Supply'] * (10 ** 6)
    Energy['Country'] = Energy['Country'].str.replace(r' \(.*\)', '', regex=True).replace(r'\d+$', '', regex= True)
    Energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States",
                    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
    ############################
    
    GDP = pd.read_csv(r'assets/world_bank.csv', skiprows=4)
    GDP['Country Name'].replace({
    "Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
    , inplace=True)
    GDP.rename({'Country Name' : 'Country'}, axis='columns', inplace=True)
    ############################
    
    ScimEn = pd.read_excel(r'assets/scimagojr-3.xlsx')
    ############################

    dfs = [ScimEn, Energy, GDP]
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='Country', how='inner'), dfs)
    merged_df.set_index('Country', inplace=True)
    merged_df = merged_df.loc[merged_df['Rank'] <16, 
                             ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
                              'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
                              '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', 
                              '2013', '2014', '2015']]
    return merged_df

In [146]:
Energy = pd.read_excel(r'assets/Energy Indicators.xls', skiprows=17, skipfooter=38).drop(columns=['Unnamed: 0', 'Unnamed: 1'])
Energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy.replace('...', np.NaN, inplace=True)
Energy['Energy Supply'] = Energy['Energy Supply'] * (10 ** 6)
Energy['Country'] = Energy['Country'].str.replace(r' \(.*\)', '', regex=True).replace(r'\d+$', '', regex= True)
Energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States",
                "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
############################

GDP = pd.read_csv(r'assets/world_bank.csv', skiprows=4)
GDP['Country Name'].replace({
"Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
, inplace=True)
GDP.rename({'Country Name' : 'Country'}, axis='columns', inplace=True)
############################

ScimEn = pd.read_excel(r'assets/scimagojr-3.xlsx')
############################

dfs = [ScimEn, Energy, GDP]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Country', how='inner'), dfs)
merged_df.shape

(161, 70)

In [3]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrameunique"
assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"

In [4]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [5]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [6]:
def answer_two():
    
    return 161 - 15

In [7]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [172]:
def answer_three():
    merged_df = answer_one()
    merged_df = merged_df.loc[:, [str(x) for x in range(2006, 2016)]]
    ans = merged_df.mean(axis= 'columns').sort_values(ascending=False)
    ans.name = 'avgGPD'
    return ans
    

In [173]:
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGPD, dtype: float64

In [9]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [10]:
def answer_four():
    avg_gpd = answer_three().reset_index()
    country_name = avg_gpd['Country'].loc[5]
    df = answer_one()
    diff = df.loc[country_name, "2015"] - df.loc[country_name, "2006"]  
    return diff

In [11]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [12]:
def answer_five():
    df = answer_one()['Energy Supply per Capita']
    return df.mean()

In [13]:
# Cell for autograder.


In [14]:
type(answer_five())

numpy.float64

### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [15]:
def answer_six():
    df = answer_one()['% Renewable'].reset_index()
    df.sort_values('% Renewable', ascending=False, inplace=True)
    x = df.iloc[0]
    return x.loc['Country'], x.loc['% Renewable']

In [16]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [17]:
def answer_seven():
    # YOUR CODE HERE
    df = answer_one()
    df['ratio'] = df['Self-citations'] / df['Citations']
    df = df['ratio'].sort_values(ascending=False).reset_index()
    return df.iloc[0, 0], df.iloc[0, 1]

In [18]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [19]:
def answer_eight():
    df = answer_one()
    frame = df[['Energy Supply', 'Energy Supply per Capita']]
    frame['pop'] = frame['Energy Supply'] / frame['Energy Supply per Capita']
    frame.sort_values('pop', ascending=False, inplace=True)
    return frame.iloc[2].name

In [20]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [21]:
def answer_nine():
    df = answer_one()
    df['pop'] = df['Energy Supply'] / df['Energy Supply per Capita']
    col_list = ['Citable documents', 'pop', 'Energy Supply per Capita']
    df = df[col_list]
    df['Citable Documents per Capita'] = df['Citable documents'] / df['pop']
    corr = df['Citable Documents per Capita'].corr(df['Energy Supply per Capita'])
    return corr

In [22]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [23]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [24]:
def answer_ten():
    df = answer_one()['% Renewable'].to_frame()
    med = np.median(df)
    df['HighRenew'] = np.where(df['% Renewable'] >= med, 1, 0)
    return df['HighRenew']

In [25]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [184]:
def answer_eleven():
    df = answer_one()
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    df['Continent'] = ContinentDict
    df['pop'] = df['Energy Supply'] / df['Energy Supply per Capita']
    agg_dict = {'Continent' : 'size', 'pop' : [np.sum, np.mean, np.std] }
    x = df.groupby('Continent').agg(agg_dict).droplevel(level=0, axis=1)
    return x

In [185]:
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [186]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [120]:
def answer_twelve():
    df = answer_one()
    bins = pd.cut(df['% Renewable'], bins= 5)
    ContinentDict  = {'China':'Asia', 
                          'United States':'North America', 
                          'Japan':'Asia', 
                          'United Kingdom':'Europe', 
                          'Russian Federation':'Europe', 
                          'Canada':'North America', 
                          'Germany':'Europe', 
                          'India':'Asia',
                          'France':'Europe', 
                          'South Korea':'Asia', 
                          'Italy':'Europe', 
                          'Spain':'Europe', 
                          'Iran':'Asia',
                          'Australia':'Australia', 
                          'Brazil':'South America'}
    df['Continent'] = ContinentDict
    s = df.groupby(['Continent', bins]).size()
    return s

In [121]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"


In [122]:
answer_twelve()

Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [181]:
def answer_thirteen():
    df = answer_one()
    df = (df['Energy Supply'] / df['Energy Supply per Capita']).apply('{:,}'.format)
    df.name = 'PopEst'
    return df

In [182]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


In [183]:
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
plot_optional()